In [9]:
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
tokenizer = PreTrainedTokenizerFast.from_pretrained('Soyoung97/gec_kr')
model = BartForConditionalGeneration.from_pretrained('Soyoung97/gec_kr',device_map="auto")
text = '한국어는어렵다.'
raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
corrected_ids = model.generate(torch.tensor([input_ids]),
                                max_length=128,
                                eos_token_id=1, num_beams=4,
                                early_stopping=True, repetition_penalty=2.0)
output_text = tokenizer.decode(corrected_ids.squeeze().tolist(), skip_special_tokens=True)
output_text

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


'한국어는 어렵다.'

In [3]:

import pandas as pd
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, Trainer
from datasets import Dataset
import re
import torch
import json
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# 데이터 로딩 함수
def load_data(file_path):
   return pd.read_json(file_path, encoding='utf-8-sig')

# 데이터 로딩
train = load_data('resource/data/대화맥락추론_train.json')
dev = load_data('resource/data/대화맥락추론_dev.json')
test = load_data('resource/data/대화맥락추론_test.json')

In [5]:
def make_chat(inp):
        chat = ["[Conversation]"]
        for cvt in inp['conversation']:
            speaker = cvt['speaker']
            utterance = cvt['utterance']
            chat.append(f"화자{speaker}: {utterance}")
        chat = "\n".join(chat)

        question = f"[Question]\n위 대화의 {inp['category']}"
        if (ord(inp['category'][-1]) - ord("가")) % 28 > 0:
            question += "으로"
        else:
            question = "로"
        question += " 올바른 지문은?"
                
        chat = chat + "\n\n" + question + "\n\n[Option]\n"
        chat += f"A. {inp['inference_1']}\n"
        chat += f"B. {inp['inference_2']}\n"
        chat += f"C. {inp['inference_3']}"

        return chat

In [6]:

dev_preprocess = [make_chat(inp) for inp in dev['input']]
train_preprocess = [make_chat(inp) for inp in train['input']]
test_preprocess = [make_chat(inp) for inp in test['input']]

In [24]:

def grammatical_error_correction(text):

    raw_input_ids = tokenizer.encode(text)

    if len(raw_input_ids) < 510:
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
        corrected_ids = model.generate(torch.tensor([input_ids]),
                                        max_length=512,
                                        eos_token_id=1, num_beams=4,
                                        early_stopping=True, repetition_penalty=2.0)
        output_text = tokenizer.decode(corrected_ids.squeeze().tolist(), skip_special_tokens=True)

    # 길이가 충분하지 않은 경우 전처리 진행 x --> 데이터 손실 방지 차원   
    else: 
        output_text = text
    

    return output_text

In [ ]:
dev_gec = [] 


for i in range(len(dev_preprocess)):
    if i % 10 == 0: 
        print(i)
    dev_gec.append(grammatical_error_correction(dev_preprocess[i]))


In [31]:
dev_preprocess[70]

'[Conversation]\n화자1: 쇼핑 자주 하시나요?\n화자2: 자주하는 편은 아니구\n화자2: 필요할 때만 해요\n화자1: 최근엔 뭐 사셨어요?\n화자2: 모기 물린데에 붙일 밴드 샀어요 마스크랑\n화자1: 모기 물린데 붙이는 밴드가 따로 있나요?\n화자2: 따로 있는건 아니구\n화자2: 긁지 못하게 막을 용도로 샀어요\n화자1: 아~ 어디꺼 사셨나요?\n화자2: 데일밴드요ㅋㅋ 가장 기본적인게 좋죠\n화자1: 아!! 모기 물린데 데일밴드 붙일 생각을 저는 왜 못했을까요??\n화자2: ㅋㅋ 저도 이번에 심하게 물리고 나서 생각했어요\n화자1: 긁다 피나잖아요~ 저도 밴드 사야겠어요 ㅎ\n화자2: ㅎㅎ 좋아요\n화자1: 전 오늘 택배 박스만 4개 받았어요 ㅋㅋ\n\n[Question]\n위 대화의 후행사건으로 올바른 지문은?\n\n[Option]\nA. 화자1도 모기에 심하게 물렸으므로 밴드를 구매할 것이다.\nB. 화자1도 모기에 물렸을 때를 대비해 밴드를 구매할 것이다.\nC. 화자1도 진드기에 물렸을 때를 대비해 밴드를 구매할 것이다.'

In [32]:
dev_gec[70]

'[Conversation]\n화자1: 쇼핑 자주 하시나요?\n화자2: 자주하는 편은 아니구\n 화자2: 필요할 때만 사셨어요?\n화자2: 모기 물린데 데일밴드를 샀어요'

In [19]:
dev_gec = [grammatical_error_correction(inp) for inp in dev_preprocess]




In [20]:

train_gec = [grammatical_error_correction(inp) for inp in train_preprocess]

In [21]:
dev_gec 

[('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.', 1),
 ('한국어는 어렵다.